Initialization

In [328]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

train_data = pd.read_csv("./train.csv")
val_data = pd.read_csv("./test.csv")

print(train_data.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [329]:
from sklearn import preprocessing

In [330]:
y = train_data["Survived"]
print(y)

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64


In [331]:
train_data["Sex"] = train_data["Sex"].apply(lambda sex: 0 if sex == "male" else 1)

In [332]:
X = train_data.drop(["Survived", "PassengerId", "Name", "Ticket", "Cabin", "Embarked"], axis=1)
X = X.interpolate(method="linear", axis=0)

In [333]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,0,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,3,1,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,3,0,35.0,0,0,8.0500


In [334]:
def norm(x: float | int, df: pd.DataFrame, label: str) -> float:
    return (x - df[label].min())/(df[label].max() - df[label].min()) * 100

In [335]:
X["Age"] = X["Age"].apply(lambda x: norm(x, X, "Age"))
X["Fare"] = X["Fare"].apply(lambda x: norm(x, X, "Fare"))

In [336]:
print(y.size)

891


In [337]:
X.sample(10)

,Pclass,Sex,Age,SibSp,Parch,Fare
297,1,1,1.985423,1,2,29.580590
261,3,0,3.242021,4,2,6.126432
767,3,1,37.798442,0,0,1.512699
376,3,1,27.117366,0,0,1.415106
18,3,1,38.426740,1,0,3.513366
711,1,0,44.709726,0,0,5.182215
262,1,0,64.815280,1,1,15.546645
480,3,0,10.781603,5,2,9.154270
869,3,0,4.498618,1,1,2.173075
233,3,1,5.755215,4,2,6.126432


In [338]:
from sklearn.ensemble import RandomForestClassifier

In [339]:
?RandomForestClassifier

Init signature:
RandomForestClassifier(
    n_estimators=100,
    *,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='sqrt',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=None,
    ccp_alpha=0.0,
    max_samples=None,
)
Docstring:     
A random forest classifier.

A random forest is a meta estimator that fits a number of decision tree
classifiers on various sub-samples of the dataset and uses averaging to
improve the predictive accuracy and control over-fitting.
The sub-sample size is controlled with the `max_samples` parameter if
`bootstrap=True` (default), otherwise the whole dataset is used to build
each tree.

Read more in the :ref:`User Guide <forest>`.

Parameters
----------
n_estimators : int, default=100
    The number of trees in the forest.


In [340]:
model = RandomForestClassifier(n_estimators=200, random_state=0)

In [341]:
model.fit(X, y)

RandomForestClassifier(n_estimators=200, random_state=0)

In [342]:
test_X = val_data.drop(["PassengerId", "Name", "Ticket", "Cabin", "Embarked"], axis=1)
test_X["Sex"] = test_X["Sex"].apply(lambda sex: 0 if sex == "male" else 1)
test_X = test_X.interpolate(method="linear", axis=0)
test_X["Age"] = test_X["Age"].apply(lambda x: norm(x, test_X, "Age"))
test_X["Fare"] = test_X["Fare"].apply(lambda x: norm(x, test_X, "Fare"))

In [343]:
X.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.352413,36.825912,0.523008,0.381594,6.285843
std,0.836071,0.477990,17.469657,1.102743,0.806057,9.699511
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,25.860769,0.000000,0.000000,1.544007
50%,3.000000,0.000000,35.285248,0.000000,0.000000,2.821272
75%,3.000000,1.000000,47.222920,1.000000,0.000000,6.050797
max,3.000000,1.000000,100.000000,8.000000,6.000000,100.000000


In [344]:
test_X.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,0.363636,39.397667,0.447368,0.392344,6.942016
std,0.841838,0.481622,17.806552,0.896760,0.981429,10.902077
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,28.128709,0.000000,0.000000,1.541158
50%,3.000000,0.000000,36.304893,0.000000,0.000000,2.821272
75%,3.000000,1.000000,51.041804,1.000000,0.000000,6.142901
max,3.000000,1.000000,100.000000,8.000000,9.000000,100.000000


In [345]:
pred_y = model.predict(test_X)

In [346]:
submission = pd.DataFrame({
    "PassengerId": val_data["PassengerId"],
    "Survived": pred_y
})
submission.to_csv("./submission.csv", index=False)